In [1]:
# https://github.com/scipy/scipy/issues/7151
# https://apps.dtic.mil/sti/pdfs/AD1004183.pdf
# https://www.codeproject.com/Articles/21282/Compute-Permanent-of-a-Matrix-with-Ryser-s-Algorit

# https://rosettacode.org/wiki/Determinant_and_permanent
# https://codegolf.stackexchange.com/questions/97060/calculate-the-permanent-as-quickly-as-possible

# https://stackoverflow.com/questions/38738835/generating-gray-codes
# https://qiita.com/b1ueskydragon/items/75cfee42541ea723080c

# https://qiita.com/phdax/items/3064de264c7933bab2f5
# https://web.archive.org/web/20190108235115/https://www.hackersdelight.org/hdcodetxt/pop.c.txt
# http://graphics.stanford.edu/~seander/bithacks.html#CountBitsSetParallel
# https://stackoverflow.com/questions/9829578/fast-way-of-counting-non-zero-bits-in-positive-integer

# https://stackoverflow.com/questions/22227595/convert-integer-to-binary-array-with-suitable-padding

In [2]:
import numpy as np

from itertools import permutations
from operator import mul
from math import fsum
from functools import reduce

In [3]:
n = 5
for i in range(0, 1<<n):
    gray = i^(i>>1)
    print("{:4d}".format(i),"{0:0{1}b}".format(gray,n),"{:4d}".format(gray))

   0 00000    0
   1 00001    1
   2 00011    3
   3 00010    2
   4 00110    6
   5 00111    7
   6 00101    5
   7 00100    4
   8 01100   12
   9 01101   13
  10 01111   15
  11 01110   14
  12 01010   10
  13 01011   11
  14 01001    9
  15 01000    8
  16 11000   24
  17 11001   25
  18 11011   27
  19 11010   26
  20 11110   30
  21 11111   31
  22 11101   29
  23 11100   28
  24 10100   20
  25 10101   21
  26 10111   23
  27 10110   22
  28 10010   18
  29 10011   19
  30 10001   17
  31 10000   16


In [4]:
def count_bits(bit):
    count = bit
## 32 bits
#    count = (count & 0x55555555) + ((count >> 1) & 0x55555555)
#    count = (count & 0x33333333) + ((count >> 2) & 0x33333333)
#    count = (count & 0x0F0F0F0F) + ((count >> 4) & 0x0F0F0F0F)
#    count = (count & 0x00FF00FF) + ((count >> 8) & 0x00FF00FF)
#    count = (count & 0x0000FFFF) + ((count >>16) & 0x0000FFFF)
## 64 bits
    count = (count & 0x5555555555555555) + ((count & 0xAAAAAAAAAAAAAAAA) >> 1)
    count = (count & 0x3333333333333333) + ((count & 0xCCCCCCCCCCCCCCCC) >> 2)
    count = (count & 0x0F0F0F0F0F0F0F0F) + ((count & 0xF0F0F0F0F0F0F0F0) >> 4)
    count = (count & 0x00FF00FF00FF00FF) + ((count & 0xFF00FF00FF00FF00) >> 8)
    count = (count & 0x0000FFFF0000FFFF) + ((count & 0xFFFF0000FFFF0000) >> 16)
    count = (count & 0x00000000FFFFFFFF) + ((count & 0xFFFFFFFF00000000) >> 32)
    return count

In [5]:
def gray_code(i,n):
    gray = i^(i>>1)
    bit_gray = np.array(list(np.binary_repr(gray).zfill(n))+[count_bits(gray)]).astype(np.int64)
    return gray, count_bits(gray), bit_gray

In [6]:
n = 5
for i in range(0, 1<<n):
    gray, nbit, bit_gray = gray_code(i,n)
    print("{:4d}".format(i),"{0:0{1}b}".format(gray,n),"{:4d}".format(gray),nbit,bit_gray)

   0 00000    0 0 [0 0 0 0 0 0]
   1 00001    1 1 [0 0 0 0 1 1]
   2 00011    3 2 [0 0 0 1 1 2]
   3 00010    2 1 [0 0 0 1 0 1]
   4 00110    6 2 [0 0 1 1 0 2]
   5 00111    7 3 [0 0 1 1 1 3]
   6 00101    5 2 [0 0 1 0 1 2]
   7 00100    4 1 [0 0 1 0 0 1]
   8 01100   12 2 [0 1 1 0 0 2]
   9 01101   13 3 [0 1 1 0 1 3]
  10 01111   15 4 [0 1 1 1 1 4]
  11 01110   14 3 [0 1 1 1 0 3]
  12 01010   10 2 [0 1 0 1 0 2]
  13 01011   11 3 [0 1 0 1 1 3]
  14 01001    9 2 [0 1 0 0 1 2]
  15 01000    8 1 [0 1 0 0 0 1]
  16 11000   24 2 [1 1 0 0 0 2]
  17 11001   25 3 [1 1 0 0 1 3]
  18 11011   27 4 [1 1 0 1 1 4]
  19 11010   26 3 [1 1 0 1 0 3]
  20 11110   30 4 [1 1 1 1 0 4]
  21 11111   31 5 [1 1 1 1 1 5]
  22 11101   29 4 [1 1 1 0 1 4]
  23 11100   28 3 [1 1 1 0 0 3]
  24 10100   20 2 [1 0 1 0 0 2]
  25 10101   21 3 [1 0 1 0 1 3]
  26 10111   23 4 [1 0 1 1 1 4]
  27 10110   22 3 [1 0 1 1 0 3]
  28 10010   18 2 [1 0 0 1 0 2]
  29 10011   19 3 [1 0 0 1 1 3]
  30 10001   17 2 [1 0 0 0 1 2]
  31 100

In [7]:
def slow_gray_code(n,dim):
    res = np.zeros(dim+1,dtype=np.int64)
    pos = dim - 1
    while n>0:
        res[pos] = n%2
        res[dim] += res[pos]
        n = n//2
        pos = pos-1
    return res

In [8]:
n = 5
for i in range(0, 1<<n):
    gray = slow_gray_code(i,n)
    print(gray)

[0 0 0 0 0 0]
[0 0 0 0 1 1]
[0 0 0 1 0 1]
[0 0 0 1 1 2]
[0 0 1 0 0 1]
[0 0 1 0 1 2]
[0 0 1 1 0 2]
[0 0 1 1 1 3]
[0 1 0 0 0 1]
[0 1 0 0 1 2]
[0 1 0 1 0 2]
[0 1 0 1 1 3]
[0 1 1 0 0 2]
[0 1 1 0 1 3]
[0 1 1 1 0 3]
[0 1 1 1 1 4]
[1 0 0 0 0 1]
[1 0 0 0 1 2]
[1 0 0 1 0 2]
[1 0 0 1 1 3]
[1 0 1 0 0 2]
[1 0 1 0 1 3]
[1 0 1 1 0 3]
[1 0 1 1 1 4]
[1 1 0 0 0 2]
[1 1 0 0 1 3]
[1 1 0 1 0 3]
[1 1 0 1 1 4]
[1 1 1 0 0 3]
[1 1 1 0 1 4]
[1 1 1 1 0 4]
[1 1 1 1 1 5]


In [9]:
def fast_gray_code(i,n):
    gray = i^(i>>1)
    return np.array(list(np.binary_repr(gray).zfill(n))+[count_bits(gray)]).astype(np.int64)

In [10]:
def _gray_code(n):
    if n == 0:
        return [0]
    if n == 1:
        return [0, 1]
    codes = _gray_code(n - 1)
    for k in reversed(codes):
        codes.append((1 << (n - 1)) + k)
    return codes

In [11]:
print(_gray_code(5))

[0, 1, 3, 2, 6, 7, 5, 4, 12, 13, 15, 14, 10, 11, 9, 8, 24, 25, 27, 26, 30, 31, 29, 28, 20, 21, 23, 22, 18, 19, 17, 16]


In [12]:
n = 5
for i in range(0, 1<<n):
    print(slow_gray_code(i,n),fast_gray_code(i,n))

[0 0 0 0 0 0] [0 0 0 0 0 0]
[0 0 0 0 1 1] [0 0 0 0 1 1]
[0 0 0 1 0 1] [0 0 0 1 1 2]
[0 0 0 1 1 2] [0 0 0 1 0 1]
[0 0 1 0 0 1] [0 0 1 1 0 2]
[0 0 1 0 1 2] [0 0 1 1 1 3]
[0 0 1 1 0 2] [0 0 1 0 1 2]
[0 0 1 1 1 3] [0 0 1 0 0 1]
[0 1 0 0 0 1] [0 1 1 0 0 2]
[0 1 0 0 1 2] [0 1 1 0 1 3]
[0 1 0 1 0 2] [0 1 1 1 1 4]
[0 1 0 1 1 3] [0 1 1 1 0 3]
[0 1 1 0 0 2] [0 1 0 1 0 2]
[0 1 1 0 1 3] [0 1 0 1 1 3]
[0 1 1 1 0 3] [0 1 0 0 1 2]
[0 1 1 1 1 4] [0 1 0 0 0 1]
[1 0 0 0 0 1] [1 1 0 0 0 2]
[1 0 0 0 1 2] [1 1 0 0 1 3]
[1 0 0 1 0 2] [1 1 0 1 1 4]
[1 0 0 1 1 3] [1 1 0 1 0 3]
[1 0 1 0 0 2] [1 1 1 1 0 4]
[1 0 1 0 1 3] [1 1 1 1 1 5]
[1 0 1 1 0 3] [1 1 1 0 1 4]
[1 0 1 1 1 4] [1 1 1 0 0 3]
[1 1 0 0 0 2] [1 0 1 0 0 2]
[1 1 0 0 1 3] [1 0 1 0 1 3]
[1 1 0 1 0 3] [1 0 1 1 1 4]
[1 1 0 1 1 4] [1 0 1 1 0 3]
[1 1 1 0 0 3] [1 0 0 1 0 2]
[1 1 1 0 1 4] [1 0 0 1 1 3]
[1 1 1 1 0 4] [1 0 0 0 1 2]
[1 1 1 1 1 5] [1 0 0 0 0 1]


In [13]:
def calc_perm_slow(A,n):
    sum = 0
    rowsumprod = 0
    rowsum = 0
    chi = np.zeros(n+1,dtype=np.int64)
    C = 2**n
    for k in range(1,C):
        rowsumprod = 1
        chi = slow_gray_code(k,n)
        for m in range(n):
            rowsum = 0
            for p in range(n):
                #rowsum += chi[p] * A[m * n + p]
                rowsum += chi[p] * A[m,p]
            rowsumprod *= rowsum
        sum += (-1)**(n-chi[n]) * rowsumprod
    return sum

In [14]:
def prod(lst):
    return reduce(mul, lst, 1)

def perm(a):
    n = len(a)
    r = range(n)
    s = permutations(r)
    return fsum(prod(a[i][sigma[i]] for i in r) for sigma in s)

In [15]:
def npperm(M):
    n = M.shape[0]
    d = np.ones(n,dtype=np.float64)
    j =  0
    s = 1
    f = np.arange(n)
    v = M.sum(axis=0,dtype=np.float64)
    p = np.prod(v)
    while (j < n-1):
        v -= 2.0*d[j]*M[j]
        d[j] = -d[j]
        s = -s
        prod = np.prod(v)
        p += s*prod
        f[0] = 0
        f[j] = f[j+1]
        f[j+1] = j+1
        j = f[0]
    return p/2**(n-1) 

In [16]:
for a in (
        [
         [1, 2],
         [3, 4]],

        [
         [1, 2, 3, 4],
         [4, 5, 6, 7],
         [7, 8, 9, 10],
         [10, 11, 12, 13]],

        [
         [ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24]],
    ):
    print("")
    print(calc_perm_slow(np.array(a),np.array(a).shape[0]))
    print(perm(a))
    print(npperm(np.array(a)))


10
10.0
10.0

29556
29556.0
29556.0

6778800
6778800.0
6778800.0


In [17]:
def _calc_perm_slow(A,n):
    sum = 0
    rowsumprod = 0
    rowsum = 0
    chi = np.zeros(n+1,dtype=np.int64)
    C = 2**n
    for k in range(1,C):
        rowsumprod = 1
        chi = fast_gray_code(k,n)
        for m in range(n):
            rowsum = 0
            for p in range(n):
                #rowsum += chi[p] * A[m * n + p]
                rowsum += chi[p] * A[m,p]
            rowsumprod *= rowsum
        sum += (-1)**(n-chi[n]) * rowsumprod
    return sum

In [18]:
def _fast_gray_code(n):
    return [i^(i>>1) for i in range(0, 1<<n)]

print(_fast_gray_code(3))
print(_fast_gray_code(4))
print(_fast_gray_code(5))

def _test_npperm(M):
    n = M.shape[0]
    d = np.ones(n,dtype=np.float64)
    j =  0
    s = 1
    #f = np.arange(n)
    f = _fast_gray_code(n)
    v = M.sum(axis=0,dtype=np.float64)
    p = np.prod(v)
#    while (j < n-1):
    for j in range(n):
        v -= 2.0*f[j]*M[j]
        #v -= 2.0*d[j]*M[j]
        #d[j] = -d[j]
        s = -s
        prod = np.prod(v)
        p += s*prod
#        f[0] = 0
#        f[j] = f[j+1]
#        f[j+1] = j+1
#        j = f[0]
    return p/2**(n-1) 

[0, 1, 3, 2, 6, 7, 5, 4]
[0, 1, 3, 2, 6, 7, 5, 4, 12, 13, 15, 14, 10, 11, 9, 8]
[0, 1, 3, 2, 6, 7, 5, 4, 12, 13, 15, 14, 10, 11, 9, 8, 24, 25, 27, 26, 30, 31, 29, 28, 20, 21, 23, 22, 18, 19, 17, 16]


In [19]:
for a in (
        [
         [1, 2],
         [3, 4]],

        [
         [1, 2, 3, 4],
         [4, 5, 6, 7],
         [7, 8, 9, 10],
         [10, 11, 12, 13]],

        [
         [ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24]],
    ):
    print("")
    print(_calc_perm_slow(np.array(a),np.array(a).shape[0]))
    print(perm(a))
    print(_test_npperm(np.array(a)))


10
10.0
2.0

29556
29556.0
4841088.0

6778800
6778800.0
361934779500.0


In [20]:
j = 0
n = 5
f = np.arange(n)
while (j < n-1):
    print(j)
    f[0] = 0
    f[j] = f[j+1]
    f[j+1] = j+1
    j = f[0]

0
1
0
2
0
1
0
3
0
1
0
2
0
1
0
